# 5400 PROJECT: Military Expenditure Search Engine

## Part A: Back-end Data Warehouse

#### A-1 Preparation

In [1]:
import csv
import pandas as pd
import json
import requests
import os 
from pprint import pprint

#### A-2: Connection to MongoDB

In [2]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.apan5400

In [3]:
db.military_expenditure_share_gdp.drop()
db.military_expenditure_total.drop()
db.military_spending_per_capita.drop()
db.armed_forces_personnel.drop()
db.armed_forces_personnel_percent.drop()
db.armed_forces_personnel_of_total_labor_force.drop()

#### A-3: Import data from csv file to mongoDB

make sure path is correct when importing the data

In [4]:

def mongoimport(csv_path, coll_name):
    client = MongoClient('localhost',27017)
    db = client.apan5400
    collection = db[coll_name]
    data = pd.read_csv(csv_path)
    gdp = json.loads(data.to_json(orient='records'))
    collection.remove
    collection.insert_many(gdp)
    return collection.count

mongoimport('/Users/shayshay/Documents/5400FinalProejct/military-expenditure-share-gdp.csv', 'military_expenditure_share_gdp')
mongoimport('/Users/shayshay/Documents/5400FinalProejct/military-expenditure-total.csv', 'military_expenditure_total')
mongoimport('/Users/shayshay/Documents/5400FinalProejct/military-spending-per-capita.csv', 'military_spending_per_capita')
mongoimport('/Users/shayshay/Documents/5400FinalProejct/armed-forces-personnel.csv', 'armed_forces_personnel')
mongoimport('/Users/shayshay/Documents/5400FinalProejct/armed-forces-personnel-percent.csv', 'armed_forces_personnel_percent')
mongoimport('/Users/shayshay/Documents/5400FinalProejct/armed-forces-personnel-of-total-labor-force.csv', 'armed_forces_personnel_of_total_labor_force')
print("Done!")

Done!


In [5]:
coll_expshare = db.military_expenditure_share_gdp
coll_exptotal = db.military_expenditure_total
coll_exppercap = db.military_spending_per_capita
coll_armperson = db.armed_forces_personnel
coll_armpersent = db.armed_forces_personnel_percent
coll_armoflabor = db.armed_forces_personnel_of_total_labor_force

total_docs1 = coll_expshare.count_documents({})
total_docs2 = coll_exptotal.count_documents({})
total_docs3 = coll_exppercap.count_documents({})
total_docs4 = coll_armperson.count_documents({})
total_docs5 = coll_armpersent.count_documents({})
total_docs6 = coll_armoflabor.count_documents({})

#count on documents
print(total_docs1)
print(total_docs2)
print(total_docs3)
print(total_docs4)
print(total_docs5)
print(total_docs6)

7684
8278
4798
6444
5078
6029


## Part B: Data ETL Process Pipeline (Query Pipelines)

#### B-1: Share of GDP

In [6]:
country_input = "Afghanistan"
year_input = 1970
pipeline1 = [
 {
     "$match":{
         "$and":[
             {"Entity": {"$regex": country_input, "$options": "i"}},
             {"Year": year_input}
             ]}},
{"$group": {"_id":{
 "GDP Share of Military Expenditure" : "$military_expenditure_share_gdp"
}
 }}]

list(db.military_expenditure_share_gdp.aggregate(pipeline1))

[{'_id': {'GDP Share of Military Expenditure': 1.63}}]

#### B-2: Total Military Expenditure

In [7]:
pipeline2 = [
 {
     "$match":{
         "$and":[
             {"Entity": {"$regex": country_input,"$options": "i"}},
             {"Year": year_input}
             ]}},
{"$group": {"_id":{
 "Total Military Expenditure" : "$military_expenditure"
}
 }}]
list(db.military_expenditure_total.aggregate(pipeline2))

[{'_id': {'Total Military Expenditure': 5373185}}]

#### B-3: Military Spending Per Capita

In [8]:
year_input = 2004
country_input = "Afghanistan"
pipeline3 = [
 {
     "$match":{
         "$and":[
             {"Entity": {"$regex": country_input,"$options": "i"}},
             {"Year": year_input}
             ]}},
{"$group": {"_id":{
 "Military Spending per Capita" : "$military_expenditure_per_capita"
}
 }}]
list(db.military_spending_per_capita.aggregate(pipeline3))

[{'_id': {'Military Spending per Capita': 5.06}}]

#### B-4: Armed Forces Personnel

In [9]:
pipeline4 = [
 {
     "$match":{
         "$and":[
             {"Entity": {"$regex": country_input,"$options": "i"}},
             {"Year": year_input}
             ]}},
{"$group": {"_id":{
 "Armed Forces Personnel" : "$Armed forces personnel_total"
}
 }}]
list(db.armed_forces_personnel.aggregate(pipeline4))

[{'_id': {'Armed Forces Personnel': 27000}}]

#### B-5: Armed Forces Personnel Percent

In [10]:
pipeline5 = [
 {
     "$match":{
         "$and":[
             {"Entity": {"$regex": country_input,"$options": "i"}},
             {"Year": year_input}
             ]}},
{"$group": {"_id":{
 "Armed Forces Share Population" : "$armed_forces_share_population"
}
 }}]
list(db.armed_forces_personnel_percent.aggregate(pipeline5))

[{'_id': {'Armed Forces Share Population': 0.1091937542}}]

#### B-6: Armed Forces Personnel of Total Labor Force

In [11]:
pipeline6 = [
 {
     "$match":{
         "$and":[
             {"Entity": {"$regex": country_input,"$options": "i"}},
             {"Year": year_input}
             ]}},
{"$group": {"_id":{
 "Armed Forces Personnel of Total Labor Force" : "$Armed_forces_personnel_of total labor force"
}
 }}]
list(db.armed_forces_personnel_of_total_labor_force.aggregate(pipeline6))

[{'_id': {'Armed Forces Personnel of Total Labor Force': 0.435599375}}]

## Part C: Create Flask-API and connecting with MongoDB

Connecting with query pipelines:

In [12]:
from flask import Flask, jsonify, request
from flask_restful import Resource, Api
app = Flask(__name__)
api = Api(app)


class Hello(Resource):
    def get(self):
        return jsonify({'message': 'Hello, World!'})

class Test(Resource):
    def get(self,country_year):
        country_input = country_year.split('_')[0]
        year_input = int(country_year.split('_')[1])
        res_all = []

        pipeline1 = [
            {
                "$match":{
                    "$and":[
                        {"Entity": {"$regex": country_input, "$options": "i"}},
                        {"Year": year_input}]}},
                {"$group": {"_id":{
                    "GDP Share of Military Expenditure" : "$military_expenditure_share_gdp"}}}]
        response1=list(db.military_expenditure_share_gdp.aggregate(pipeline1))[0]['_id']
        res_all.append(response1)

        pipeline2 = [
            {
                "$match":{
                    "$and":[
                        {"Entity": {"$regex": country_input, "$options": "i"}},
                        {"Year": year_input}]}},
                {"$group": {"_id":{
                    "Total Military Expenditure" : "$military_expenditure"}}}]
        response2=list(db.military_expenditure_total.aggregate(pipeline2))[0]['_id']
        res_all.append(response2)

        pipeline3 = [
            {
                "$match":{
                    "$and":[
                        {"Entity": {"$regex": country_input, "$options": "i"}},
                        {"Year": year_input}]}},
                {"$group": {"_id":{
                    "Military Spending per Capita" : "$military_expenditure_per_capita"}}}]
        response3=list(db.military_spending_per_capita.aggregate(pipeline3))[0]['_id']
        res_all.append(response3)

        pipeline4 = [
            {
                "$match":{
                    "$and":[
                        {"Entity": {"$regex": country_input, "$options": "i"}},
                        {"Year": year_input}]}},
                {"$group": {"_id":{
                    "Armed Forces Personnel" : "$Armed forces personnel_total"}}}]
        response4=list(db.armed_forces_personnel.aggregate(pipeline4))[0]['_id']
        res_all.append(response4)

        pipeline5 = [
            {
                "$match":{
                    "$and":[
                        {"Entity": {"$regex": country_input, "$options": "i"}},
                        {"Year": year_input}]}},
                {"$group": {"_id":{
                    "Armed Forces Share Population" : "$armed_forces_share_population"}}}]
        response5=list(db.armed_forces_personnel_percent.aggregate(pipeline5))[0]['_id']
        res_all.append(response5)

        pipeline6 = [
            {
                "$match":{
                    "$and":[
                        {"Entity": {"$regex": country_input, "$options": "i"}},
                        {"Year": year_input}]}},
                {"$group": {"_id":{
                    "Armed Forces Personnel of Total Labor Force" : "$Armed_forces_personnel_of total labor force"}}}]
        response6=list(db.armed_forces_personnel_of_total_labor_force.aggregate(pipeline6))[0]['_id']
        res_all.append(response6)


        return jsonify(res_all)
print("Done!")

#Defined resources along with their corresponding urls
api.add_resource(Hello, '/')
api.add_resource(Test, '/square/<string:country_year>')

# driver function
app.run(host='localhost', port=5001)

Done!
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5001 (Press CTRL+C to quit)
127.0.0.1 - - [02/May/2022 16:58:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 16:58:36] "GET /square/china_2004 HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 16:58:55] "GET /square/china_1998 HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 16:59:02] "GET /square/japan_2008 HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 16:59:10] "GET /square/italy_2010 HTTP/1.1" 200 -


Open on the http, there will appear: {"message":"Hello, World!"}

If change the url to http://localhost:5001/square/japan_2010

Then, Japan's military data will appear as a list containing dictionary objects:

[{"GDP Share of Military Expenditure":0.96},{"Total Military Expenditure":46420031665},{"Military Spending per Capita":425.19},{"Armed Forces Personnel":260382},{"Armed Forces Share Population":0.2025651484},{"Armed Forces Personnel of Total Labor Force":0.390561135}]

## PART D: Graphical User Interface  (GUI)

- use tkinter package to create GUI.
- flask-api is required to run, before running GUI. Please don't close connection.

### The GUI code is on a separate file. Please open the notebook called 2GUI.ipynb